# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [ ]:
#!pip install chromadb langchain langchain_community langchain_openai

In [1]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [ ]:
# If you're using colab, run this
#os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

In [3]:
llm = OpenAI(temperature=0)

In [7]:
# from pathlib import Path

# relevant_parts = []
# for p in Path(".").absolute().parts:
#     relevant_parts.append(p)
#     if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
#         break
# # doc_path = str(Path(*relevant_parts) / "state_of_the_union.txt")
# doc_path = str(Path(*relevant_parts) / "sonnets.txt")

doc_path = "sonnets.txt"


In [9]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
# docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [14]:
# Let's see what we just created and estimate token usage
print(f"Number of text chunks created: {len(texts)}")
print(f"Approximate characters in all chunks: {sum(len(str(chunk)) for chunk in texts)}")
print("This many API calls were made to OpenAI for embeddings!")

Number of text chunks created: 154
Approximate characters in all chunks: 103052
This many API calls were made to OpenAI for embeddings!


In [13]:
docsearch

In [10]:
sonnets = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [11]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [12]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [16]:
docs = loader.load()

# Check the size of what we loaded
total_characters = sum(len(doc.page_content) for doc in docs)
total_pages = len(docs)

print(f"Number of documents loaded: {total_pages}")
print(f"Total characters: {total_characters}")
print(f"Approximate tokens (characters ÷ 4): {total_characters // 4}")
print(f"Approximate cost for embeddings: ${(total_characters / 1000) * 0.00013:.6f}") # This model uses text-embedding-3-large will be priced at $0.00013 / 1k tokens.

# Now see how many chunks we'll create
ruff_texts = text_splitter.split_documents(docs)
print(f"Number of chunks that will be embedded: {len(ruff_texts)}")

Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 2321, which is longer than the specified 1000


Number of documents loaded: 1
Total characters: 23945
Approximate tokens (characters ÷ 4): 5986
Approximate cost for embeddings: $0.003113
Number of chunks that will be embedded: 22


In [17]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 2321, which is longer than the specified 1000
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


## Create the Agent

In [18]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [ ]:
tools = [
    Tool(
        name="Shakespeare Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about Shakespeare's sonnets. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [23]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [24]:
agent.invoke("What is the main theme of Shakespeare's sonnets?")



> Entering new AgentExecutor chain...
 The main theme of Shakespeare's sonnets is a common question, so there should be a tool to help answer it.
Action: Shakespeare Sonnets QA System
Action Input: "What is the main theme of Shakespeare's sonnets?"

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given



Observation:  The main theme of Shakespeare's sonnets is love, particularly the speaker's love for a young man, often referred to as the "fair youth."
Thought: This answer seems accurate, but it would be helpful to have another source to confirm.
Action: Ruff QA System
Action Input: "What is the main theme of Shakespeare's sonnets?"

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given



Observation:  I don't know.
Thought: It seems like Ruff QA System is not the best tool for answering this type of question.
Action: Shakespeare Sonnets QA System
Action Input: "What is the main theme of Shakespeare's sonnets?"

In [40]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 You should always think about the differences between ruff and flake8
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation:  Ruff can replace multiple Flake8 plugins, has a larger rule set, and can automatically fix its own lint violations. It also supports Python versions from 3.7 onwards and does not require the installation of Rust. Additionally, Ruff can be used independently as a linter or formatter, and can be used alongside a type checker for more comprehensive error detection.
Thought: I now know the final answer
Final Answer: Ruff offers more features and flexibility compared to flake8, making it a better choice for linting and formatting Python code.

> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': 'Ruff offers more features and flexibility compared to flake8, making it a better choice for linting and formatting Python code.'}

In [ ]:
# agent.invoke(
#     "What did biden say about ketanji brown jackson in the state of the union address?"
# )

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question
Action: State of Union QA System
Action Input: What did biden say about ketanji brown jackson in the state of the union address?
Observation:  Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, praising her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.
Thought: I now know the final answer
Final Answer: Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, praising her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.

> Finished chain.


"Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, praising her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence."

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [25]:
# tools = [
#     Tool(
#         name="State of Union QA System",
#         func=state_of_union.run,
#         description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
#         return_direct=True,
#     ),
#     Tool(
#         name="Ruff QA System",
#         func=ruff.run,
#         description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
#         return_direct=True,
#     ),
# ]

tools = [
    Tool(
        name="Shakespeare Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about Shakespeare's sonnets. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [26]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [48]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question
Action: State of Union QA System
Action Input: What did biden say about ketanji brown jackson in the state of the union address?
Observation:  Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, praising her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.


> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': " Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, praising her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence."}

In [50]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 You should always think about the differences between ruff and flake8
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation:  Ruff can replace multiple Flake8 plugins, has a larger rule set, and can automatically fix its own lint violations. It also supports Python versions from 3.7 onwards and does not require the installation of Rust. Additionally, Ruff can be used independently as a linter or formatter, and can be used alongside a type checker for more comprehensive error detection.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': ' Ruff can replace multiple Flake8 plugins, has a larger rule set, and can automatically fix its own lint violations. It also supports Python versions from 3.7 onwards and does not require the installation of Rust. Additionally, Ruff can be used independently as a linter or formatter, and can be used alongside a type checker for more comprehensive error detection.'}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [27]:
# tools = [
#     Tool(
#         name="State of Union QA System",
#         func=state_of_union.run,
#         description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
#     ),
#     Tool(
#         name="Ruff QA System",
#         func=ruff.run,
#         description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
#     ),
# ]


tools = [
    Tool(
        name="Shakespeare Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about Shakespeare's sonnets. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [55]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [57]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
 I should use the Ruff QA System to answer the first question and the State of Union QA System to answer the second question.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff integrates with nbQA, a tool for running linters and code formatters over Jupyter Notebooks.
Thought: I should use the State of Union QA System to answer the second question.
Action: State of Union QA System
Action Input: Did the president mention that tool in the state of the union?
Observation: 
No, the president did not mention a specific tool in the state of the union.
Thought: I now know the final answer.
Final Answer: No, the president did not mention a specific tool in the state of the union.

> Finished chain.


{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?',
 'output': 'No, the president did not mention a specific tool in the state of the union.'}